# PEGパーサを作る

## クラス設計

In [3]:
from abc import ABC

class PExpr(ABC):
    pass

class PChar(PExpr):
    text: str
    def __init__(self, text):
        self.text = text

assert PChar('')    # ''
assert PChar('a')   # a
assert PChar('ab')  # ab


class PAny(PExpr):
    pass

assert PAny()  # .

class PNot(PExpr):
    e: PExpr
    def __init__(self, e: PExpr):
        self.e = e

assert PNot(PChar('a'))  # !a
assert PNot(PNot(PChar('a')))  # &a

class PSeq(PExpr):
    e1: PExpr
    e2: PExpr
    def __init__(self, e1: PExpr, e2: PExpr):
        self.e1 = e1
        self.e2 = e2

assert PSeq(PChar('a'), PChar('b'))
assert PSeq(PChar('a'), PSeq(PChar('b'), PChar('c')))

class POre(PExpr):
    e1: PExpr
    e2: PExpr
    def __init__(self, e1: PExpr, e2: PExpr):
        self.e1 = e1
        self.e2 = e2

assert POre(PChar('a'), PChar('b'))
assert POre(PChar('a'), POre(PChar('b'), PChar('c')))

class PRef(PExpr):
    peg: dict # PEG 文法
    name: str # 非終端記号の名前 
    def __init__(self, peg, name):
        self.peg = peg
        self.name = name

peg = {}
peg['A'] = PRef(peg, 'B') 
peg['B'] = PChar('b')

## パーザインタプリタ

In [4]:
from abc import ABC, abstractmethod
class PExpr(ABC):
    @abstractmethod
    def match(self, x):
        pass # 抽象メソッドなので何もしない

### PChar :文字マッチのクラス定義

In [5]:
class PChar(PExpr):
    text: str
    def __init__(self, text):
        self.text = text
    def match(self, x):
        if x.startswith(self.text):
            y = x[len(self.text):]
            return y
        return None


PChar('a').match('ab')

'b'

### PAny :任意 1 文字マッチのクラス定義

In [6]:
class PAny(PExpr):
    def match(self, x):
        if len(x) > 0:
            y = x[1:]
            return y
        return None

PAny().match('ab')

'b'

### PNot :否定のクラス定義

In [7]:
class PNot(PExpr):
    e: PExpr
    def __init__(self, e: PExpr):
        self.e = e
    def match(self, x):
        o = self.e.match(x)
        if o is None:
            return x
        return None

PNot(PChar('b')).match('ab')

'ab'

In [8]:
class PSeq(PExpr):
    e1: PExpr
    e2: PExpr
    def __init__(self, e1, e2):
        self.e1 = e1
        self.e2 = e2
    def match(self, x):
        y = self.e1.match(x)
        if y is None:
            return None
        return self.e2.match(y)

e = PSeq(PChar('a'), PChar('b')) 
e.match('ab')

''

In [9]:
class POre(PExpr):
    e1: PExpr
    e2: PExpr
    def __init__(self, e1, e2):
        self.e1 = e1
        self.e2 = e2
    def match(self, x):
        y = self.e1.match(x)
        if y != None:
            return y
        return self.e2.match(x)


e = POre(PChar('a'), PChar('b')) 
e.match('ab')

'b'

In [10]:
e = POre(PChar('b'), PChar('a')) 
e.match('ab')

'b'

### PRef: 非終端記号

In [11]:
class PRef(PExpr):
    peg: dict
    name: str
    def __init__(self, peg, name):
        self.peg = peg
        self.name = name
    def match(self, x):
        e = self.peg[self.name]
        return e.match(x)

peg = {}
peg["A"] = PRef(peg, "B") 
peg["B"] = PChar('b')

peg['A'].match('bb')

'b'

In [12]:
peg = {}
peg["A"] = POre(PSeq(PChar('a'), PRef(peg, 'A')), PChar(''))

peg['A'].match('aaaabb')

'bb'

### 9.4.1 コンストラクタ関数

In [13]:
def POption(e):
    if isinstance(e, str):
        e = PChar(e)
    return POre(e, PChar(''))

POption('a').match('ab')

'b'

In [14]:
POption('a').match('ba')

'ba'

In [15]:
def PRange(text):
    if len(text) == 1:
        return PChar(text)
    return POre(PChar(text[0]), PRange(text[1:]))

PRange('ab').match('ba')

'a'

### 9.4.2 繰り返し

In [21]:
class PMany(PExpr):
    e: PExpr
    def __init__(self, e):
        self.e = e
    def match(self, x):
        y = self.e.match(x)
        while y != None:
            x = y
            y = self.e.match(x)
        return x

In [22]:
PMany(PChar('a')).match('aaaabb')

'bb'

In [23]:
PMany(PChar('a')).match('bbaaaa')

'bbaaaa'

### PEG 演算子のオーバーロード

上位クラス(PExpr)を修正したら、サブクラスも再度、実行して定義し直す。

In [24]:
class PExpr(object):
    def __truediv__(self, e):
        if isinstance(e, str):
            e = PChar(e)
        return POre(self, e)

class PRef(PExpr):
    peg: dict
    name: str
    def __init__(self, peg, name):
        self.peg = peg
        self.name = name
    def match(self, x):
        e = self.peg[self.name]
        return e.match(x)

peg = {}
peg["A"] = PRef(peg, "B") / 'a' 
peg["B"] = PChar('b')